## Music store (Chinook database) analysis

### Question 1:
* 1.1 How many customers does the store have in total?
* 1.2 How many invoices (orders) are there in total?
* 1.3 Over how many distinct countries are the customers spread?


In [9]:
#1.1 How many customers does the store have in total?
import pandas as pd

df = pd.read_csv('./data/Chinhook_data/Customer.csv')
df['CustomerId'].count()

np.int64(59)

In [12]:
#1.2 How many invoices (orders) are there in total?
df2 = pd.read_csv('./data/Chinhook_data/Invoice.csv')
df2['InvoiceId'].count()

np.int64(412)

In [18]:
#1.3 Over how many distinct countries are the customers spread?
countries = df2['BillingCountry'].unique()
print(f"There are {len(countries)} unique countries.")

There are 24 unique countries.


### Question 2 – Sales by country and customer
* 2.1a What are the top 5 countries by total sales amount?
* 2.1b For each of those 5 countries, what is the total sales and the number of customers?
* 2.2 For each of those top 5 countries, who is the highest-spending customer (full name) and what is their total spending?

In [27]:
df2.columns

Index(['InvoiceId', 'CustomerId', 'InvoiceDate', 'BillingAddress',
       'BillingCity', 'BillingState', 'BillingCountry', 'BillingPostalCode',
       'Total'],
      dtype='object')

In [54]:
#2.1a What are the top 5 countries by total sales amount?
df = pd.read_csv('./data/Chinhook_data/Invoice.csv')
top5 = (
    df.groupby('BillingCountry')['Total']
      .sum()
      .sort_values(ascending=False)
      .head(5)
      .reset_index(name='Total_expense')   # new column name
)

print(top5)


  BillingCountry  Total_expense
0            USA         523.06
1         Canada         303.96
2         France         195.10
3         Brazil         190.10
4        Germany         156.48


In [ ]:
#2.1b For each of those 5 countries, what is the total sales and the number of customers?
df = pd.read_csv('./data/Chinhook_data/Invoice.csv')

result = (
    df.groupby('BillingCountry')
      .agg(
          Total_expense=('Total', 'sum'),
          num_customers=('CustomerId', 'nunique')
      )
      .sort_values('Total_expense', ascending=False)
      .head(5)
      .reset_index()
)

print(result)


  BillingCountry  Total_expense  num_customers
0            USA         523.06             13
1         Canada         303.96              8
2         France         195.10              5
3         Brazil         190.10              5
4        Germany         156.48              4


In [ ]:
#2.2 For each of those top 5 countries, who is the highest-spending customer (full name) and what is their total spending?

In [31]:
import pandas as pd

df = pd.read_csv('./data/Chinhook_data/Invoice.csv')

df1 = (df.groupby('BillingCountry')
        .agg(Total_Expense=('Total','sum'),
             )
        .sort_values('Total_Expense', ascending=False)
        .head(5)
        )

df2 = df1.merge(df, how='inner', on='BillingCountry')
df2
# 1) Sum Total per (BillingCountry, CustomerId)
df3 = (df2.groupby(['BillingCountry','CustomerId'])
       .agg(total_expense=('Total', 'sum'))
    )
df3
# 2) For each country, keep the customer with highest total_expense
idx = agg.groupby('BillingCountry')['total_expense'].idxmax()
top_customers = agg.loc[idx].reset_index(drop=True)
top_customers

df = pd.read_csv('./data/Chinhook_data/Customer.csv')

df4 = top_customers.merge(df, how='inner', on='CustomerId')
df4 = df4[['CustomerId','FirstName','LastName','BillingCountry','total_expense']]
df4

,CustomerId,FirstName,LastName,BillingCountry,total_expense
0,1,Luís,Gonçalves,Brazil,39.62
1,3,François,Tremblay,Canada,39.62
2,43,Isabelle,Mercier,France,40.62
3,37,Fynn,Zimmermann,Germany,43.62
4,26,Richard,Cunningham,USA,47.62


In [35]:
import pandas as pd

# Load data
invoices = pd.read_csv('./data/Chinhook_data/Invoice.csv')
customers = pd.read_csv('./data/Chinhook_data/Customer.csv')

# (Optional) Top 5 countries by total invoice amount
top_countries = (
    invoices.groupby('BillingCountry', as_index=False)
            .agg(country_expense=('Total', 'sum'))
            .nlargest(5, 'country_expense')
)

# Keep only those countries (remove this block if you want all countries)
invoices_top = invoices.merge(top_countries[['BillingCountry']], on='BillingCountry')

# Total purchase per (country, customer)
country_customer_totals = (
    invoices_top.groupby(['BillingCountry', 'CustomerId'], as_index=False)
                .agg(total_expense=('Total', 'sum'))
)

# For each country, customer with highest total purchase
idx = country_customer_totals.groupby('BillingCountry')['total_expense'].idxmax()
top_customers = country_customer_totals.loc[idx].reset_index(drop=True)

# Attach customer names
result = (
    top_customers
    .merge(customers[['CustomerId', 'FirstName', 'LastName']], on='CustomerId')
    [['CustomerId', 'FirstName', 'LastName', 'BillingCountry', 'total_expense']]
)
result

,CustomerId,FirstName,LastName,BillingCountry,total_expense
0,1,Luís,Gonçalves,Brazil,39.62
1,3,François,Tremblay,Canada,39.62
2,43,Isabelle,Mercier,France,40.62
3,37,Fynn,Zimmermann,Germany,43.62
4,26,Richard,Cunningham,USA,47.62


### Q3.1.
* Which 5 tracks generated the highest total sales revenue (sum of unitprice * quantity)?
* For each of these tracks, list: track name, artist name, album title, total units sold, and total revenue.


In [1]:
import pandas as pd

df = pd.read_csv('./data/Chinhook_dataset/InvoiceLine.csv')
df['sales_revenue'] = df['UnitPrice']*df['Quantity']
df_revenue = df.sort_values('sales_revenue',ascending=False)
df_revenue.head(5)


,InvoiceLineId,InvoiceId,TrackId,UnitPrice,Quantity,sales_revenue
2199,2200,404,2922,1.99,1,1.99
2198,2199,404,2913,1.99,1,1.99
2197,2198,404,2904,1.99,1,1.99
2196,2197,404,2895,1.99,1,1.99
2195,2196,404,2886,1.99,1,1.99


In [67]:
df_track = (
    df_revenue
    .groupby(["TrackId",'UnitPrice'], as_index=False)
    .agg(
        total_units=("Quantity","sum"),
        total_revenue=("sales_revenue","sum")
    )
)
df_track=df_track.sort_values('total_revenue',ascending=False)
df_track.head()

,TrackId,UnitPrice,total_units,total_revenue
1843,3223,1.99,2,3.98
1656,2868,1.99,2,3.98
1840,3214,1.99,2,3.98
1823,3177,1.99,2,3.98
1830,3200,1.99,2,3.98


In [72]:
Track_info= pd.read_csv('./data/Chinhook_dataset/Track.csv')
df_track.merge(Track_info[['TrackId', "Name", 'AlbumId']], on="TrackId").rename(columns={'Name':"track_name"})


,TrackId,UnitPrice,total_units,total_revenue,track_name,AlbumId
0,3223,1.99,2,3.98,How to Stop an Exploding Man,228
1,2868,1.99,2,3.98,Walkabout,230
2,3214,1.99,2,3.98,Phyllis's Wedding,251
3,3177,1.99,2,3.98,Hot Girl,249
4,3200,1.99,2,3.98,Gay Witch Hunt,251
...,...,...,...,...,...,...
1979,1209,0.99,1,0.99,Lord of Light,94
1980,1204,0.99,1,0.99,The Pilgrim,94
1981,1203,0.99,1,0.99,Brighter Than a Thousand Suns,94
1982,1202,0.99,1,0.99,These Colours Don't Run,94


### Q3.2.
* For each genre, compute the total units sold and total revenue worldwide.
* From that result, identify the top 3 genres by revenue.


In [7]:
import pandas as pd
df_genre = pd.read_csv('./data/Chinhook_dataset/Genre.csv')
df_genre.columns

Index(['GenreId', 'Name'], dtype='object')

In [8]:
df_track = pd.read_csv('./data/Chinhook_dataset/Track.csv')
#df_track.head()

In [9]:
df_megred_temp = df_genre.merge(df_track,on='GenreId',how='inner', suffixes=('Genre','Track')).drop(columns=['Bytes','Milliseconds','MediaTypeId','AlbumId','UnitPrice'])
#df_megred_temp.head()

In [10]:
df_invoiceLine = pd.read_csv('./data/Chinhook_dataset/InvoiceLine.csv')
df_megred= df_megred_temp.merge(df_invoiceLine, on='TrackId', how='inner').drop(columns=['InvoiceId','InvoiceLineId'])
#df_megred.head()

In [ ]:
df_megred['Revenue']=df_megred['UnitPrice'] * df_megred['Quantity']
ans =(df_megred.groupby('NameGenre')
      .agg(
          Total_units_sold=('Quantity','sum'),
          Total_revenue=('Revenue','sum')
      ).sort_values(by='Total_revenue',ascending=False)
      .head(3)
      )
ans

,Total_units_sold,Total_revenue
NameGenre,,
Rock,835,826.65
Latin,386,382.14
Metal,264,261.36


In [ ]:
# # AI answer: cleaner
# import pandas as pd

# # 1. Load data (update paths as needed)
# invline = pd.read_csv("InvoiceLine.csv")
# track   = pd.read_csv("Track.csv")
# genre   = pd.read_csv("Genre.csv")

# # 2. Compute line revenue
# invline["Revenue"] = invline["UnitPrice"] * invline["Quantity"]

# # 3. Map tracks to genres
# track_genre = track[["TrackId", "GenreId"]].merge(genre, on="GenreId")  # genre has columns GenreId, Name

# # 4. Fact table: one row per invoice line + genre
# fact = invline.merge(track_genre, on="TrackId")

# # 5. Aggregate per genre
# genre_sales = (
#     fact
#     .groupby(["GenreId", "Name"], as_index=False)
#     .agg(UnitsSold=("Quantity", "sum"),
#          Revenue=("Revenue", "sum"))
#     .sort_values("Revenue", ascending=False)
# )

# # 6. Top 3 genres by revenue
# top3 = genre_sales.head(3)

# print(genre_sales)   # full Q3.2 result: units + revenue for every genre
# print("\nTop 3 genres by revenue:\n", top3)


### Q3.3.
* Focusing only on USA customers, which genre has the highest number of tracks sold?
* For that top USA genre, which 3 artists have the highest track sales (units sold) in the USA?


In [98]:
import pandas as pd
df = pd.read_csv('./data/Chinhook_dataset/Invoice.csv')
usa_customers = df[df['BillingCountry'] == 'USA'].drop(columns=['InvoiceDate','BillingPostalCode','CustomerId'],axis=1)
invoice_line =pd.read_csv('./data/Chinhook_dataset/InvoiceLine.csv')
full_usa_cust=usa_customers[['InvoiceId','BillingCountry']].merge(invoice_line[['InvoiceId','TrackId','UnitPrice','Quantity']], on='InvoiceId')
track =pd.read_csv('./data/Chinhook_dataset/Track.csv')
usa_genre_cust = full_usa_cust.merge(track[['TrackId','GenreId']], on='TrackId')
genre =pd.read_csv('./data/Chinhook_dataset/Genre.csv')
df = usa_genre_cust.merge(genre, on='GenreId')
df_finall = (df.groupby('Name')
             .agg(
                 Total_sales=('Quantity','sum')
             )
             .sort_values('Total_sales', ascending=False)
             .head(1))
df_finall

,Total_sales
Name,
Rock,157


In [95]:
#Second part
df1 = df[df['Name']=='Rock']
df2 = df1.merge(track[['TrackId','AlbumId']], on='TrackId' )
album =pd.read_csv('./data/Chinhook_dataset/Album.csv')
df3 = df2.merge(album[['AlbumId','ArtistId']], on='AlbumId')
df3
top_3 = (df3.groupby(['ArtistId'])
         .agg(
             Total_sales = ('Quantity','sum')
         )
         .sort_values(by='Total_sales', ascending=False)
         .head(3))
artist =pd.read_csv('./data/Chinhook_dataset/Artist.csv')
top_3_name = top_3.merge(artist,on='ArtistId')
top_3_name

,ArtistId,Total_sales,Name
0,150,23,U2
1,58,15,Deep Purple
2,90,14,Iron Maiden


In [ ]:
#Answer with AI
import pandas as pd

# Load data
cust   = pd.read_csv("./data/Chinhook_dataset/Customer.csv")
inv    = pd.read_csv("./data/Chinhook_dataset/Invoice.csv")
invline = pd.read_csv("./data/Chinhook_dataset/InvoiceLine.csv")
track  = pd.read_csv("./data/Chinhook_dataset/Track.csv")
genre  = pd.read_csv("./data/Chinhook_dataset/Genre.csv")
album  = pd.read_csv("./data/Chinhook_dataset/Album.csv")
artist = pd.read_csv("./data/Chinhook_dataset/Artist.csv")

# 1) USA customers and their invoices
cust_usa = cust[cust["Country"] == "USA"]
inv_usa = inv[inv["CustomerId"].isin(cust_usa["CustomerId"])]
usa_lines = invline[invline["InvoiceId"].isin(inv_usa["InvoiceId"])]

# 2) Attach genre information to USA lines
track_genre = track[["TrackId", "GenreId"]].merge(genre, on="GenreId")
usa_fact = usa_lines.merge(track_genre, on="TrackId")

# 3) Find the top USA genre by units
by_genre_usa = (
    usa_fact
    .groupby(["GenreId", "Name"], as_index=False)["Quantity"]
    .sum()
    .sort_values("Quantity", ascending=False)
)
top_genre_id = by_genre_usa.iloc[0]["GenreId"]

# 4) Restrict to that top genre (Rock)
usa_top_genre = usa_fact[usa_fact["GenreId"] == top_genre_id]

# 5) Map tracks -> albums -> artists
track_album = track[["TrackId", "AlbumId"]].merge(
    album[["AlbumId", "ArtistId"]], on="AlbumId"
)

usa_top_genre_art = usa_top_genre.merge(track_album, on="TrackId")

# 6) Aggregate units per artist within that genre in the USA
by_artist_usa_top_genre = (
    usa_top_genre_art
    .groupby("ArtistId", as_index=False)["Quantity"]
    .sum()
    .sort_values("Quantity", ascending=False)
)

# 7) Attach artist names and take top 3
top3_artists_usa_top_genre = (
    by_artist_usa_top_genre
    .merge(artist, on="ArtistId")
    .head(3)
)

print(by_genre_usa.head())              # shows Rock as top USA genre
print(top3_artists_usa_top_genre)       # 3 artists with highest units in that genre


   GenreId                Name  Quantity
0        1                Rock       157
6        7               Latin        91
2        3               Metal        64
3        4  Alternative & Punk        50
1        2                Jazz        22
   ArtistId  Quantity         Name
0       150        23           U2
1        58        15  Deep Purple
2        90        14  Iron Maiden
